In [7]:
import asyncio
from pyinjective.async_client import AsyncClient
from pyinjective.core.network import Network
import pandas as pd
from pyinjective.client.model.pagination import PaginationOption
from functions import marketId_to_ticker,extract_eth_address,map_eth_to_inj,mapper

network = Network.local()
client = AsyncClient(network)


async def get_positions(client,market_ids):
    skip = 0
    limit = 100
    array = [market_ids]
    pagination = PaginationOption(skip=skip, limit=limit)
    response = await client.fetch_derivative_positions_v2(market_ids=array[0], pagination=pagination)
    positions = pd.DataFrame([order for order in response['positions']])
    return positions

ticker = "TRUMP/USDT PERP"

market_id = await marketId_to_ticker(ticker)
df = await get_positions(client,market_id)
df["entryPrice"] = df["entryPrice"].astype(float)/10**6
df["margin"] = df["margin"].astype(float)/10**6  
df["liquidationPrice"] = df["liquidationPrice"].astype(float)/10**6
df["markPrice"] = df["markPrice"].astype(float)

df["Dollars_size_$"] = df["quantity"].astype(float)*df["markPrice"]

columns_to_remove = ['marketId', 'updatedAt','updatedAt','denom']
df = df.drop(columns=columns_to_remove)

df['subaccountId'] = df['subaccountId'].apply(extract_eth_address)
eth_addresses_array = df['subaccountId'].unique()

eth_to_inj_dict = {}

await mapper(eth_addresses_array,eth_to_inj_dict)

df['subaccountId'] = df['subaccountId'].map(eth_to_inj_dict)

df["quantity"]=df["quantity"].astype(float)

df.sort_values("quantity",ascending=False,inplace=True)
df

,ticker,subaccountId,direction,quantity,entryPrice,margin,liquidationPrice,markPrice,Dollars_size_$
8,TRUMP/USDT PERP,inj15v3zuznyj0lgttnqh720t9vpp2va4jq5c672sw,long,1536.40,31.630492,48836.646894,-0.173247,0.0,0.0
18,TRUMP/USDT PERP,inj1akxycslq8cjt0uffw4rjmfm3echchptu52a2dq,short,1256.71,37.765068,16965.672954,46.604671,0.0,0.0
7,TRUMP/USDT PERP,inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z,short,1171.65,37.965122,11140.603638,43.157814,0.0,0.0
12,TRUMP/USDT PERP,inj1hny8rk4455rkynj44l42jdss5sjvg3kvv58tdh,long,962.55,37.309635,36208.402972,-0.341701,0.0,0.0
9,TRUMP/USDT PERP,inj14k7wgey85pt226ugr9sq4h2ya774n82hzy26nf,short,121.79,37.538639,1524.975016,45.509079,0.0,0.0
2,TRUMP/USDT PERP,inj19c27ec7p0dlycsv8ptxzxsef50en2eg782y06u,long,16.50,42.570000,500.990822,13.563244,0.0,0.0
6,TRUMP/USDT PERP,inj10s4hsuu4g7mujya0x6g9j5ng3q2nfgdwxfgq3c,long,15.21,40.184300,239.244314,27.172101,0.0,0.0
4,TRUMP/USDT PERP,inj1w2y98zraw0flxvly3eeszz4yax63nfne06kk9f,long,8.75,54.203061,474.614429,-0.042875,0.0,0.0
15,TRUMP/USDT PERP,inj1uxkh2ddx4d4wjpj7vj0nk8dzm06v9yfwaxhl9h,long,3.00,53.746667,136.553820,9.143030,0.0,0.0
17,TRUMP/USDT PERP,inj1a05c44d346actm53gfuurvd3x0w9hcsa89vsff,long,2.50,39.550000,39.505150,26.386600,0.0,0.0
